In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
%matplotlib inline

C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
import scipy as sp
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from sklearn.ensemble import AdaBoostRegressor
from tsfresh.utilities.dataframe_functions import impute

In [ ]:
df = pd.read_csv("data/rsbuddy/buy_average.csv")
df = df.set_index('timestamp')
df2 = pd.read_csv("data/rsbuddy/sell_average.csv")
df2 = df2.set_index('timestamp')

In [ ]:
dropped = df.drop_duplicates()
dropped2 = df2.drop_duplicates()
selected = dropped[['Chaos_rune', 'Nature_rune', 'Soul_rune', 'Blood_rune', 'Law_rune', 'Rune_axe']]
selected = selected.replace(to_replace=0, method='ffill')
selected2 = dropped2[['Chaos_rune', 'Nature_rune', 'Soul_rune', 'Blood_rune', 'Law_rune']]
# y = runes.reset_index()['Chaos_rune']

In [ ]:
print(selected.shape)
print(selected2.shape)
selected.head()

# Using the tsfresh library

In [ ]:
selected['Rune_axe'].plot(figsize=(15, 6))
plt.show()

In [ ]:
df_shift, y = make_forecasting_frame(selected["Rune_axe"], kind="buy_average", max_timeshift=20, rolling_direction=1)

In [ ]:
df_shift.head()

In [ ]:
df_shift.shape

In [ ]:
%%capture
X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute, show_warnings=False)

In [ ]:
# drop constant features
print(X.shape)
X = X.loc[:, X.apply(pd.Series.nunique) != 1] 
print(X.shape)

In [ ]:
# Add last value as feature
X["feature_last_value"] = y.shift(1)

In [ ]:
# Drop first line
X = X.iloc[1:, ]
y = y.iloc[1: ]

In [ ]:
X.head()

In [ ]:

from tqdm import tqdm

In [ ]:
ada = AdaBoostRegressor(n_estimators=10)
y_pred = [np.NaN] * len(y)

isp = 100   # index of where to start the predictions
assert isp > 0

for i in tqdm(range(isp, len(y))):
    
    ada.fit(X.iloc[:i], y[:i])
    y_pred[i] = ada.predict(X.iloc[i, :].values.reshape((1, -1)))[0]
    
y_pred = pd.Series(data=y_pred, index=y.index)

In [ ]:

# Dataframe of predictions and true values
ys = pd.concat([y_pred, y], axis = 1).rename(columns = {0: 'pred', 'value': 'true'})

# Convert index to a datetime
ys.index = pd.to_datetime(ys.index)
ys.head()

In [ ]:
ys.plot(figsize=(15, 8))
plt.title('Predicted and True Price')
plt.show()

In [ ]:
# Create column of previous price
ys['y-1'] = ys['true'].shift(1)
ys[['y-1', 'true']].plot(figsize = (15, 8))
plt.title('Benchmark Prediction and True Price')
plt.show()

In [ ]:
print("MAE y-1: \t{}".format(np.mean(np.abs(np.diff(y))[isp-1:] )))
print("MAE ada: \t{}".format(np.mean(np.abs(y_pred - y)[isp:])))

In [ ]:
importances = pd.Series(index=X.columns, data=ada.feature_importances_)
importances.sort_values(ascending=False).head(10)

# Using known finance features

In [ ]:
def moving_average_convergence(group, nslow=26, nfast=12):
    emaslow = group.ewm(span=nslow, min_periods=1).mean()
    emafast = group.ewm(span=nfast, min_periods=1).mean()
    result = pd.DataFrame({'MACD': emafast-emaslow, 'emaSlw': emaslow, 'emaFst': emafast})
    return result

def moving_average(group, n=9):
    sma = group.rolling(n).mean()
    return sma

In [ ]:
print(selected.shape)
print(selected2.shape)
selected.head()

In [ ]:
myResult = moving_average_convergence(selected['Rune_axe'])
sma = moving_average(selected['Rune_axe'])

In [ ]:
print(myResult.shape)
myResult.head()

In [ ]:
sma.head(15)

In [ ]:
def RSI(price, n=14):
    delta = price.diff()
    dUp, dDown = delta.copy(), delta.copy()
    dUp[dUp < 0] = 0
    dDown[dDown > 0] = 0

    RolUp = dUp.rolling(n).mean()
    RolDown = dDown.rolling(n).mean().abs()
    
    RS = RolUp / RolDown
    rsi= 100.0 - (100.0 / (1.0 + RS))
    return rsi

In [ ]:
rsi = RSI(selected['Rune_axe'], 10)
rsi = rsi.rename("RSI")
rsi.head(15)

In [ ]:
selected['datetime'] = selected.index
selected['datetime'] = pd.to_datetime(selected['datetime'], unit='s')
selected['dayofweek'] = selected['datetime'].dt.dayofweek
selected['hour'] = selected['datetime'].dt.hour
selected['minute'] = selected['datetime'].dt.minute

selected.head()

In [ ]:
tmp = selected.copy()
tmp.index = pd.to_datetime(tmp.index)

slope = pd.Series(np.gradient(tmp['Rune_axe']), tmp.index, name='slope')

df = pd.concat([tmp, slope], axis=1)
df.head()

In [ ]:
from scipy import integrate

def integrate_method(self, how='trapz', unit='s'):
    '''Numerically integrate the time series.

    @param how: the method to use (trapz by default)
    @return 

    Available methods:
     * trapz - trapezoidal
     * cumtrapz - cumulative trapezoidal
     * simps - Simpson's rule
     * romb - Romberger's rule

    See http://docs.scipy.org/doc/scipy/reference/integrate.html for the method details.
    or the source code
    https://github.com/scipy/scipy/blob/master/scipy/integrate/quadrature.py
    '''
    available_rules = set(['trapz', 'cumtrapz', 'simps', 'romb'])
    if how in available_rules:
        rule = integrate.__getattribute__(how)
    else:
        print('Unsupported integration rule: %s' % (how))
        print('Expecting one of these sample-based integration rules: %s' % (str(list(available_rules))))
        raise AttributeError
    
    result = rule(self.values, self.index.astype(np.int64) / 10**9)
    #result = rule(self.values)
    return result

pd.Series.integrate = integrate_method

integral = pd.Series(tmp['Rune_axe'].integrate('cumtrapz'))
integral.head()

In [ ]:
# Recursive Feature Elimination
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
# load the iris datasets
dataset = datasets.load_iris()
# create a base classifier used to evaluate a subset of attributes
model = LogisticRegression()
# create the RFE model and select 3 attributes
rfe = RFE(model, 3)
rfe = rfe.fit(dataset.data, dataset.target)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)